# Data Load

## Procedure

In [1]:
import re
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
runtime_log = []
section_flag = 0
def log_time():
  t = time.time()
  runtime_log.append(t)
  return t
def time_flag(note = 'Process complete!', frum = -2, to = -1, save_flag = False):
  log_time()
  print(f'\n{note} ' +
        f'({np.floor((runtime_log[to] - runtime_log[frum]) / 60)} minutes and {(runtime_log[to] - runtime_log[frum]) % 60} seconds)')
  if save_flag:
    return len(runtime_log) - 1

In [3]:
log_time()
!pip install kaggle
!kaggle datasets download -d dilwong/flightprices
time_flag()

Dataset URL: https://www.kaggle.com/datasets/dilwong/flightprices
License(s): Attribution 4.0 International (CC BY 4.0)
flightprices.zip: Skipping, found more recently modified local copy (use --force to force download)

Process complete! (0.0 minutes and 5.638140439987183 seconds)


In [4]:
log_time()
!unzip -n flightprices.zip
time_flag()

Archive:  flightprices.zip

Process complete! (0.0 minutes and 0.10419678688049316 seconds)


In [5]:
log_time()
!pip install pyspark
time_flag()


Process complete! (0.0 minutes and 7.452688932418823 seconds)


In [6]:
log_time()
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
try:
  sc.stop()
except:
  pass
sc = SparkContext()
sqlContext = SQLContext(sc)
time_flag()

/usr/local/lib/python3.11/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(



Process complete! (0.0 minutes and 19.468690156936646 seconds)


In [7]:
log_time()
ss = SparkSession.builder.getOrCreate()
time_flag()


Process complete! (0.0 minutes and 0.010963678359985352 seconds)


In [8]:
log_time()
df = sqlContext.read.csv('itineraries.csv', header = True)
time_flag()


Process complete! (0.0 minutes and 24.004635334014893 seconds)


In [9]:
log_time()
df.show()
time_flag()

+--------------------+----------+----------+---------------+------------------+-------------+--------------+-----------+--------------+------------+---------+--------+---------+--------------+-------------------+---------------------------------+------------------------+-------------------------------+----------------------+--------------------------+----------------------------+--------------------+-------------------+----------------------------+-------------------------+----------------+-----------------+
|               legId|searchDate|flightDate|startingAirport|destinationAirport|fareBasisCode|travelDuration|elapsedDays|isBasicEconomy|isRefundable|isNonStop|baseFare|totalFare|seatsRemaining|totalTravelDistance|segmentsDepartureTimeEpochSeconds|segmentsDepartureTimeRaw|segmentsArrivalTimeEpochSeconds|segmentsArrivalTimeRaw|segmentsArrivalAirportCode|segmentsDepartureAirportCode| segmentsAirlineName|segmentsAirlineCode|segmentsEquipmentDescription|segmentsDurationInSeconds|segments

## Summary
(View runtime of procedure following execution)

In [10]:
section_flag = time_flag(note = 'Data Load Complete!', frum = section_flag, save_flag = True)


Data Load Complete! (0.0 minutes and 58.58775448799133 seconds)


# Data Partitioning/Preprocessing

New approach: begin by first grouping by the number of flight legs. Since this implicitly partitions the data by feature space dimension, an intuitive next step would be to train one model per partition. One must be careful to check that the distribution of partition sizes is appropriately balanced when employing this strategy.

## Procedure

In [11]:
log_time()
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.regression import *
from pyspark.ml.feature import *
time_flag()


Process complete! (0.0 minutes and 0.4747164249420166 seconds)


In [12]:
log_time()
# Count flight legs per entry.
nL = udf(lambda x: len(x.split('||')))
qcols = ['"' + c + '"' for c in df.columns]
flights_w_legs = eval(f"df.select({', '.join(qcols)}, nL('segmentsDistance').cast('int').alias('legs'))")
per_leg_stats = flights_w_legs.groupby('legs').count()
per_leg_stats.show()
time_flag()

+----+--------+
|legs|   count|
+----+--------+
|   1|22066888|
|   3| 7586488|
|   4|  199812|
|   2|52285467|
|   5|      98|
+----+--------+


Process complete! (11.0 minutes and 12.030950784683228 seconds)


In [13]:
log_time()
# Only interested in flights that are not incredibly rare with respect to number
# of legs. Given the extensive bulk of our dataset, we can use a relatively
# generous threshold, e.g., 0.05 * df.count() / (per_leg_stats.count() - 1)

thresh = 0.05 * df.count() / (per_leg_stats.count() - 1)
common_wrt_legs = list(per_leg_stats.where(col('count') > thresh).select('legs').toPandas()['legs'])
time_flag()


Process complete! (24.0 minutes and 23.339475393295288 seconds)


In [14]:
log_time()
# Train-test split
train_df, test_df = df.randomSplit([0.75, 0.25], 42)
time_flag()


Process complete! (0.0 minutes and 0.07444262504577637 seconds)


In [15]:
log_time()
# Get leg-based partitions for train/test sets...
lbp_train = {}
lbp_test = {}

flights_w_legs_train = eval(f"train_df.select({', '.join(qcols)}, nL('segmentsDistance').cast('int').alias('legs'))")
flights_w_legs_test = eval(f"test_df.select({', '.join(qcols)}, nL('segmentsDistance').cast('int').alias('legs'))")

for i in common_wrt_legs:
  lbp_train[i] = flights_w_legs_train.where(col('legs') == i)
  lbp_test[i] = flights_w_legs_test.where(col('legs') == i)

# Data has now been officially partitioned/filtered!
time_flag()


Process complete! (0.0 minutes and 0.4627115726470947 seconds)


In [16]:
log_time()
# Define abbreviations for the leg-based features.
feat_abbs = dict(zip(['segmentsDepartureTimeRaw', 'segmentsArrivalTimeRaw',
                      'segmentsArrivalAirportCode', 'segmentsDepartureAirportCode',
                     'segmentsAirlineCode', 'segmentsDurationInSeconds', 'segmentsDistance'],
                      ['sDTR_', 'sATR_', 'sAAC_', 'sDAC_', 'sAC_', 'sDIS_', 'sD_']))
feat_others = ['searchDate', 'flightDate', 'isBasicEconomy', 'seatsRemaining']
targets = ['baseFare', 'totalFare']

def leg_breaker_maker(delim, n):
  return udf(lambda x: x.split(delim)[n])

plc_train = {}
plc_test = {}
for i in common_wrt_legs:
  tmp = []
  for j in feat_abbs:
    tmp.append(", ".join([f'leg_breaker_maker("||", {k})(col("{j}").alias(str({k})))' for k in range(i)]))
  plc_train[i] = (eval(f'lbp_train[{i}].select([{", ".join(tmp)}] + feat_others + targets)'))
  plc_train[i] = eval(f'plc_train[{i}].withColumnsRenamed(dict(zip(plc_train[{i}].columns, ' +
                      f'np.concatenate([[feat_abbs[a] + str(k + 1) for k in range({i})] for a in feat_abbs]))))')
  plc_test[i] = (eval(f'lbp_test[{i}].select([{", ".join(tmp)}] + feat_others + targets)'))
  plc_test[i] = eval(f'plc_test[{i}].withColumnsRenamed(dict(zip(plc_test[{i}].columns, ' +
                      f'np.concatenate([[feat_abbs[a] + str(k + 1) for k in range({i})] for a in feat_abbs]))))')
time_flag()


Process complete! (0.0 minutes and 2.58178448677063 seconds)


In [17]:
log_time()
for i in common_wrt_legs:
  plc_train[i].show()
time_flag()

+--------------------+--------------------+------+------+-----+------+----+----------+----------+--------------+--------------+--------+---------+
|              sDTR_1|              sATR_1|sAAC_1|sDAC_1|sAC_1|sDIS_1|sD_1|searchDate|flightDate|isBasicEconomy|seatsRemaining|baseFare|totalFare|
+--------------------+--------------------+------+------+-----+------+----+----------+----------+--------------+--------------+--------+---------+
|2022-05-16T18:08:...|2022-05-16T20:00:...|   JFK|   CLT|   B6|  6720| 545|2022-04-17|2022-05-16|         False|             7|  143.26|   167.11|
|2022-05-05T09:00:...|2022-05-05T10:57:...|   DTW|   ATL|   F9|  7020| 604|2022-04-17|2022-05-05|         False|             4|   70.12|    89.98|
|2022-04-20T08:00:...|2022-04-20T09:34:...|   LGA|   BOS|   DL|  5640| 185|2022-04-16|2022-04-20|         False|             5|  591.63|   650.60|
|2022-04-21T11:00:...|2022-04-21T12:13:...|   CLT|   ATL|   DL|  4380| 228|2022-04-17|2022-04-21|         False|      

In [18]:
log_time()
# Upon preliminary preprocessing, we can make the following observations:

# sAAC, sDAC, sAC, isBasicEconomy are categorical variables--these are all to be
# one-hot encoded.

# sDIS, sD, seatsRemaining are numerical variables--these are only to be cast to
# numerical data types.

# sDTR/sATR, searchDate, flightDate are date varibles--from these, we can
# extract more nuanced features such as times of day (TOD) for
# departures/arrivals, days till flight (DTF) between search and flight dates,
# and days of weeks (DOY) and months (MOY) of flight dates.

rp_train = {}
rp_test = {}
cat_feats = {}
num_feats = {}
date_feats = {}

# Prepare lists of names for one-hot encoding of categorical features.
cat_feats_si = {}
cat_feats_ohe = {}

for i in common_wrt_legs:
  cat_feats[i] = ['isBasicEconomy'] + [c for c in plc_train[i].columns if c.startswith(('sAAC_', 'sDAC_', 'sAC_'))]
  num_feats[i] = ['seatsRemaining'] + [c for c in plc_train[i].columns if c.startswith(('sDIS_', 'sD_'))]
  date_feats[i] = ['searchDate', 'flightDate'] + [c for c in plc_train[i].columns if c.startswith(('sDTR_', 'sATR_'))]

  # Fill lists of names for one-hot encoding of categorical features.
  cat_feats_si[i] = [f + '_si' for f in cat_feats[i]]
  cat_feats_ohe[i] = [f + '_ohe' for f in cat_feats[i]]

  # Cast numerical features to numerical data types, and extract select
  # date-based features (TOD will be forgone due to the ambiguity of timezones
  # in the dataset).
  rp_train[i] = plc_train[i].select(cat_feats[i] +
   [plc_train[i][f].cast('float') for f in num_feats[i] + targets] +
    [datediff('flightDate', 'searchDate').alias('DTF')] +
     [dayofweek('flightDate').alias('DOW')] +
      [month('flightDate').alias('MOY')])
  rp_test[i] = plc_test[i].select(cat_feats[i] +
   [plc_test[i][f].cast('float') for f in num_feats[i] + targets] +
    [datediff('flightDate', 'searchDate').alias('DTF')] +
     [dayofweek('flightDate').alias('DOW')] +
      [month('flightDate').alias('MOY')])

time_flag()


Process complete! (0.0 minutes and 0.7946422100067139 seconds)


## Summary
(View runtime of procedure following execution)

In [19]:
section_flag = time_flag(note = 'Data Partitioning Complete!', frum = section_flag, save_flag = True)


Data Partitioning Complete! (36.0 minutes and 2.506894826889038 seconds)


# Partition Selection
For intricate model training, it is recommended to evaluate a small number of partitions (e.g., one or two) at a time due to computational constraints.

## Procedure

In [20]:
# Select whichever partitions are desired for evaluation.
pnos = [1, 2, 3]
fp_train = dict(zip(pnos, [rp_train[p] for p in pnos]))
fp_test = dict(zip(pnos, [rp_test[p] for p in pnos]))

## Summary
(View runtime of procedure following execution)

In [21]:
section_flag = time_flag(note = 'Partition Selection Complete!', frum = section_flag, save_flag = True)


Partition Selection Complete! (0.0 minutes and 0.0448305606842041 seconds)


# Baseline Predictions
Per partition, we evaluate the performance of the optimal trivial predictor relative to the MSE, which is simply the mean of the training labels.

## Procedure

In [22]:
target_choice = 'baseFare'

In [23]:
# Generate basline predictions, using the mean of the training labels as a
train_tm = {}
train_SE_tp = {}
test_SE_tp = {}
for j in pnos:
  log_time()
  print(f'Generating baseline performance for data partition {j}...')
  train_tm[j] = fp_train[j].agg({target_choice: 'mean'}).first().asDict()[f'avg({target_choice})']
  train_SE_tp[j] = fp_train[j].select(((fp_train[j][target_choice] - lit(train_tm[j])) ** 2).alias('SE'))
  print(f'Baseline squared error stats for train set of data partition {j}:')
  train_SE_tp[j].describe().show()
  test_SE_tp[j] = fp_test[j].select(((fp_test[j][target_choice] - lit(train_tm[j])) ** 2).alias('SE'))
  print(f'Baseline squared error stats for test set of data partition {j}:')
  test_SE_tp[j].describe().show()
  time_flag()
time_flag(note = 'Baseline squared error stats generated across all data partitions', frum = section_flag)

Generating baseline performance for data partition 1...
Baseline squared error stats for train set of data partition 1:
+-------+--------------------+
|summary|                  SE|
+-------+--------------------+
|  count|            16551316|
|   mean|  23686.754970773836|
| stddev|  165602.76281822217|
|    min|0.002997716010358...|
|    max|1.8899756398209155E7|
+-------+--------------------+

Baseline squared error stats for test set of data partition 1:
+-------+--------------------+
|summary|                  SE|
+-------+--------------------+
|  count|             5515572|
|   mean|  23633.522259205434|
| stddev|   162236.8093874404|
|    min|0.002997716010358...|
|    max|1.8899756398209155E7|
+-------+--------------------+


Process complete! (59.0 minutes and 21.833638429641724 seconds)
Generating baseline performance for data partition 2...
Baseline squared error stats for train set of data partition 2:
+-------+--------------------+
|summary|                  SE|
+-------+-

## Summary
(View runtime of procedure following execution)

In [24]:
section_flag = time_flag(note = 'Baseline Prediction Generation Complete!', frum = section_flag, save_flag = True)


Baseline Prediction Generation Complete! (176.0 minutes and 3.9943594932556152 seconds)


# Notebook Summary: Total Runtime

In [25]:
time_flag(note = 'ENTIRE NOTEBOOK EXECUTION COMPLETE!!!!!', frum = 0)


ENTIRE NOTEBOOK EXECUTION COMPLETE!!!!! (213.0 minutes and 5.142282247543335 seconds)


In [ ]:
time.sleep(12 * 60 * 60)